# Second quantised ab initio Hamiltonians with ML quantum states (VMC)
A short tutorial/overview of methods/open questions

In [ ]:
!pip install pyscf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 MB 18.8 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pyscf

from pyscf import gto, scf, ao2mo, lo

We are looking at a molecule (H2), and prepare a second quantization Hamiltonian

$$
\hat{H} = \sum_{\sigma}\sum_{ij} h^{(1)}_{ij} \hat{c}^\dagger_{j, \sigma} \hat{c}_{i, \sigma} + \sum_{ijkl} h^{(2)}_{ijkl} \sum_{\sigma, \sigma'}\frac{1}{2} \hat{c}^\dagger_{i, \sigma} \hat{c}^\dagger_{j, \sigma'} \hat{c}_{k, \sigma'} \hat{c}_{l, \sigma}
$$

To set up the Hamiltonian we need $h_{ij}$ and $h_{ijkl}$. This is generated by many ab initio packages and can be obtained in various different ways (downfolding/embedding vs tabulated atomic orbitals vs ...).

Here we get a simple H2 Hamiltonian in a minimal (tabulated) basis set with pySCF.

In [ ]:
# Set up H2 system
dist = 1.8

mol = gto.Mole()

mol.build(
    atom=[("H", (x, 0.0, 0.0)) for x in dist * np.arange(2)],
    basis="sto-6g",
    symmetry=True,
    unit="Bohr",
)

nelec = mol.nelectron
print("Number of electrons: ", nelec)

myhf = scf.RHF(mol)
ehf = myhf.scf()
norb = myhf.mo_coeff.shape[1]
print("Number of molecular orbitals: ", norb)

# Get one- and two-electron integrals for canonical basis

# 1-electron 'core' hamiltonian terms, transformed into MO basis
h1 = np.linalg.multi_dot((myhf.mo_coeff.T, myhf.get_hcore(), myhf.mo_coeff))

# Get 2-electron electron repulsion integrals, transformed into MO basis
eri = ao2mo.incore.general(myhf._eri, (myhf.mo_coeff,) * 4, compact=False)

# Previous representation exploited permutational symmetry in storage. Change this to a 4D array.
# Integrals now stored as h2[p,q,r,s] = (pq|rs) = <pr|qs>. Note 8-fold permutational symmetry.
h2 = ao2mo.restore(1, eri, norb)

Number of electrons:  2
converged SCF energy = -1.08667659340088
Number of molecular orbitals:  2


Now there are different choices to rotate the one- and two-electron tensors with unitary (single-body) transformations. This changes the definition of the underlying orthogonalized molecular orbitals and consequently the ground state wavefunction also changes in structure.

Broadly speaking there are three different choices

1.) "Canonical" orbitals (from Hartree-Fock)
  Most quantum chemistry uses these
  a) Most wavefunctions will be very peaked
  b) The Hamiltonian is dense (O[L^4] non-zero elements)

2.) "Local" orbitals (define orbitals such that they are localized on the atoms), various schemes to do the orthogonalization
  a) Most wavefunctions will be very distributed across the full Hilbert space (="dense")
  b) The Hamiltonian will get sparse asymptotically

3.) A combination of the above or other heuristics (split-local orbitals [this is what DMRG would commonly use], natural orbitals [diagonalize 1-RDM], ...)

In [ ]:
# example how to transform orbitals in pySCF

# above orbitals are already in HF (canonical orbital basis)

# Find the hamiltonian in the local basis
loc_coeff = lo.Boys(mol, myhf.mo_coeff).kernel()
hij_local = np.linalg.multi_dot((loc_coeff.T, myhf.get_hcore(), loc_coeff))
hijkl_local = ao2mo.restore(1, ao2mo.kernel(mol, loc_coeff), norb)

# split-local basis
loc_coeff_occ = lo.Boys(mol, myhf.mo_coeff[:, : nelec // 2]).kernel()
loc_coeff_vrt = lo.Boys(mol, myhf.mo_coeff[:, nelec // 2 :]).kernel()
loc_coeff = np.concatenate((loc_coeff_occ, loc_coeff_vrt), axis=1)
hij_split = np.linalg.multi_dot((loc_coeff.T, myhf.get_hcore(), loc_coeff))
hijkl_split = ao2mo.restore(1, ao2mo.kernel(mol, loc_coeff), norb)


Now the Hamiltonian is defined

-> We need a solver!

Split up in teams:
- using Netket as a solver
- using mVMC as a solver
- Openfermion -> Netket solver
- GPSKet solver

In [ ]:
solve(h1, h2)

In [ ]:

self.h_mat = h_mat
        self.eri_mat = eri_mat

        # See [Neuscamman (2013), https://doi.org/10.1063/1.4829835] for the definition of t
        self.t_mat = self.h_mat - 0.5 * np.einsum("prrq->pq", eri_mat)